In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datacompy
import os
import numpy as np

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.tree import plot_tree

# modele + Smote
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from ctgan import CTGAN

In [47]:
df = pd.read_csv("D:/ml/xgboost-main/data/ssh_logs/SSH.csv")
df = df.drop(columns=["user", "ts"])
print(df.columns)
df.head()
df = df.drop_duplicates()
df.shape
y = df["target"]
X = df.drop(columns=["target"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
df_data = df.copy()
dell = pd.concat([X_test, y_test], axis=1)      #polaczenie macierzy X_test oraz y_test
#print(dell.shape)                              #271-55=216
target_num = dell['target'].value_counts()
print(f'Klasyfikator: {target_num}')
df_cleaned = df_data.merge(dell, how='left', indicator=True)            # Dodaje kolumnę '_merge' dla identyfikacji
df_cleaned = df_cleaned[df_cleaned['_merge'] == 'left_only'].drop('_merge', axis=1)

Index(['is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count',
       'ip_failure', 'ip_success', 'no_failure', 'first', 'td', 'target'],
      dtype='object')
Klasyfikator: target
0    41
1    14
Name: count, dtype: int64


In [48]:
# concat all datasets
file_path4 = "D:\\ml\\xgboost-main\\data\\processed\\generated_data_test.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik
#df_cleaned = pd.read_csv()
smote_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\smote_data.csv")
boarderline_smote_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\boarderlinesmote_data.csv")
gan_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\GAN_data.csv")

data1 = pd.concat([df_cleaned, smote_data])
data1['source'] = ['original' if i < len(df_cleaned) else 'smote' for i in range(len(data1))]
data2 = pd.concat([boarderline_smote_data, gan_data])
data2['source'] = ['boarderline' if i < len(boarderline_smote_data) else 'gan' for i in range(len(data2))]
data = pd.concat([data1, data2])
data = data.drop_duplicates()
num_duplicates = data.duplicated().sum()
print(num_duplicates)
print(data)
#data.to_csv(file_path4,index=False)
target_num = data['target'].value_counts()
print(f'Klasyfikatory: {target_num}')
columns_list = data.columns
print('Kolumny:\n',columns_list)

y_under = data['target']
X_under = data.loc[:, ['is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count',
       'ip_failure', 'ip_success', 'no_failure', 'first', 'td']]


RUS = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = RUS.fit_resample(X_under, y_under)

print("Rozkład przed undersamplingiem:", np.bincount(y_under))
print("Rozkład po undersamplingu:", np.bincount(y_resampled))

#dodanie danych tak zeby bylo wiadomo skad pochodza, potem wylaczenie ostatniej kokumny przed undersamplingiem 
#i sprawdzenie % jaki wychodzi wspolczynnik wykorzystanych danych do najoptymalniejszej nauki

0
     is_private  is_failure  is_root  is_valid  not_valid_count  ip_failure  \
0             1           1        0         1                0           1   
1             1           1        0         1                0           2   
2             1           0        0         1                0           0   
3             1           1        0         1                0           1   
4             1           1        0         1                0           2   
..          ...         ...      ...       ...              ...         ...   
105           1           1        0         1               20          14   
106           1           1        0         1                5          26   
107           0           1        1         0                8          30   
108           1           1        1         0               28           8   
109           1           1        0         0               29          38   

     ip_success  no_failure  first   td  target  

In [55]:
data_cal = pd.concat([X_resampled, y_resampled], axis=1)
data_duplicate = data.copy()
data_duplicate = data_duplicate.drop_duplicates()
df_cal = pd.DataFrame(columns=data.columns)

In [57]:
# Łączenie danych z oznaczeniem, skąd pochodzą wiersze
#df_cal = pd.concat([df_cal, data_duplicate["source"]])
""" df_cal = pd.merge(
    df_cal,
    data_duplicate[['source']],  # Wyciągamy tylko kolumnę 'source' z data_duplicate
    how='inner'
    )

"""
print("Liczba wierszy w data_duplicate:", len(data_duplicate))

#print("Liczba wierszy po merge:", len(df_cal))
#print(df_cal) 
unique_data_cal = pd.merge(data_cal, data_duplicate, how="inner")
#unique_data_cal = unique_data_cal.drop_duplicates()
#print(unique_data_cal['source'].isnull().sum())  # Sprawdzenie braków
unique_data_cal = pd.merge(data_cal, df_cal, how='outer', indicator=True)
unique_data_cal = unique_data_cal.drop_duplicates()

# Filtrowanie wierszy występujących tylko w `data_cal`
print(unique_data_cal)

#print("Liczba wierszy w data_cal:", len())
# Dopasowanie na podstawie indeksów (lub wspólnych kolumn)
data_duplicate_indexed = data_duplicate.set_index(['is_private', 'is_failure', 'is_root', 'target'])

unique_data_cal_indexed = unique_data_cal.set_index(['is_private', 'is_failure', 'is_root', 'target'])

# Aktualizacja kolumny 'source' w unique_data_cal
unique_data_cal_indexed['source'].update(data_duplicate_indexed['source'])

# Reset indeksu, aby zachować oryginalną strukturę
unique_data_cal = unique_data_cal_indexed.reset_index()

print("Zaktualizowany unique_data_cal:")
print(unique_data_cal)

Liczba wierszy w data_duplicate: 506
     is_private  is_failure  is_root  is_valid  not_valid_count  ip_failure  \
0             1           1        0         1                0           1   
1             1           1        0         1                0           2   
2             1           0        0         1                0           0   
3             1           1        0         1                0           1   
4             1           1        0         1                0           2   
..          ...         ...      ...       ...              ...         ...   
322           1           1        1         0               13          29   
323           1           1        0         0               28          38   
324           0           1        0         1                0           5   
325           1           1        0         1               11          33   
327           1           1        0         0               14          45   

     ip_succes

ValueError: cannot handle a non-unique multi-index!